In [1]:
#!pip3 install gensim


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../data"))
import json,requests,glob
import gensim
import zipfile
import lzma
# Any results you write to the current directory are saved as output.

['lost+found']


In [8]:
!wget    ftp://ftpmirror.your.org/pub/wikimedia/dumps/frwiki/latest/frwiki-latest-pages-articles-multistream.xml.bz2 -P data
#!wget  https://dumps.wikimedia.org/frwiki/latest/frwiki-latest-pages-articles-multistream.xml.bz2 -P data

--2019-09-11 04:35:45--  ftp://ftpmirror.your.org/pub/wikimedia/dumps/frwiki/latest/frwiki-latest-pages-articles-multistream.xml.bz2
           => ‘data/frwiki-latest-pages-articles-multistream.xml.bz2’
Resolving ftpmirror.your.org (ftpmirror.your.org)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘ftpmirror.your.org’


In [3]:
import bz2
import time
def fake_tk(t,*args):
  return t.split(" " )

stime = time.time()
dump_path=glob.glob("../data/**/frwiki*.xml.bz2",recursive=True)[0]
wiki = gensim.corpora.wikicorpus.WikiCorpus(dump_path,lemmatize=False ,dictionary ={} , lower=False ,
                  token_min_len=1,tokenizer_func =fake_tk )

n=1
threshold=-1
with zipfile.ZipFile("frwiki.zip", mode="w",compression=zipfile.ZIP_DEFLATED) as zf:
    with bz2.open("../data/frwiki.txt.bz2","wt") as textfile:
          for text in wiki.get_texts():
            if n > threshold:
                text_s=' '.join(text)
                text=bytes(text_s, 'utf-8').decode('utf-8') + '\n'   
                zf.writestr(f"article{n}.txt",text)
                st=os.statvfs(".")
                textfile.write(text_s)
                if st.f_bavail*st.f_bsize<1e7:
                    break



            if n%50000==0:
                print(n)
                print(time.time()-stime)
                if n > threshold:
                    print(text[:300])      
            elif n%10000==0:
                print(n)
            n+=1
            

KeyboardInterrupt: 